# Circulant Tensor Nuclear Norm Minimization (CTNNM)

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def prox_2d(z, w, lmbda):
    N, T = z.shape
    temp1 = np.fft.fft2(lmbda * z - w) / lmbda
    temp2 = 1 - N * T / (lmbda * np.abs(temp1))
    temp2[temp2 <= 0] = 0
    return np.fft.ifft2(temp1 * temp2).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def flip_mat(X):
    temp = np.append(X, np.flip(X, axis = 1), axis = 1)
    return np.append(temp, np.flip(temp, axis = 0), axis = 0)

def inv_flip_mat(mat):
    dim = mat.shape
    N = int(dim[0] / 2)
    T = int(dim[1] / 2)
    temp = (mat[:, : T] + np.flip(mat[:, T :], axis = 1)) / 2
    return (temp[: N, :] + np.flip(temp[N :, :], axis = 0)) / 2

def CTNNM(y_true, y, lmbda, maxiter = 50):
    dim1, dim2, dim3 = y.shape
    eta = 100 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = np.zeros((2 * dim1, 2 * dim2, dim3))
    flip_y = np.zeros((2 * dim1, 2 * dim2, dim3))
    for t in range(dim3):
        flip_y_true[:, :, t] = flip_mat(y_true[:, :, t])
        flip_y[:, :, t] = flip_mat(y[:, :, t])
    N, T, L = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = np.zeros((N, T, L))
        for t in range(dim3):
            x[:, :, t] = prox_2d(z[:, :, t], w[:, :, t], lmbda)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = np.zeros((dim1, dim2, dim3))
        for t in range(dim3):
            y_hat[:, :, t] = inv_flip_mat(x[:, :, t])
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

## Large Time Series Imputation

PeMS dataset is available at https://github.com/xinychen/transdim/tree/master/datasets/California-data-set.

Hyperparameter:

- $\lambda=10^{-5}NT$.


In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('pems-w{}.npz'.format(t))['arr_0'],
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
# np.savez_compressed('dense_mat.npz', dense_mat)
# np.savez_compressed('sparse_mat.npz', sparse_mat)

# import cupy as np

# dense_mat = np.load('dense_mat.npz')['arr_0']
# sparse_mat = np.load('sparse_mat.npz')['arr_0']

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
maxiter = 100
tensor_hat = CTNNM(dense_mat, sparse_mat, lmbda, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>